<table style="width: 100%; border: none;">
  <tr style="border: none;">
    <td style="width: 15%; border: none; vertical-align: middle;">
      <img src="https://th.bing.com/th/id/OIP.zZshkvEYWRVv1orlaBMmjwHaHa?w=155&h=180&c=7&r=0&o=5&pid=1.7" alt="Logo UNALM" style="width: 100px;">
    </td>
    <td style="width: 85%; border: none; vertical-align: middle; text-align: left;">
      <h2 style="margin: 0; color: #1a4d2e;">UNIVERSIDAD NACIONAL AGRARIA LA MOLINA</h2>
      <h4 style="margin: 2px 0; color: #444;">FACULTAD DE ECONOMÍA Y PLANIFICACIÓN</h4>
      <h5 style="margin: 0; color: #666; font-weight: normal;">DEPARTAMENTO DE ESTADÍSTICA E INFORMÁTICA</h5>
    </td>
  </tr>
</table>

<hr style="border-top: 2px solid #1a4d2e; margin: 10px 0;">

<div style="padding: 10px;">
  <p style="font-size: 16px; margin: 5px 0;">
    <strong>Curso:</strong> LENGUAJE DE PROGRAMACIÓN II <br>
    <strong>Docente:</strong> Denisse Chalan
  </p>

  <h4 style="margin-top: 15px; color: #1a4d2e;">Integrantes:</h4>
  <ul style="line-height: 1.6;">
    <li>Ormeño Sakihama, Daniel Kenyi - (20240723)</li>
    <li>Apellido Paterno, Nombre - (Código)</li>
    <li>Apellido Paterno, Nombre - (Código)</li>
    <li>Apellido Paterno, Nombre - (Código)</li>
  </ul>
</div>